In [1]:
# Project setup and imports
import os, sys, random
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from svgutils.compose import Figure, Panel, SVG
import pickle
from matplotlib import colors as mcolors
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
import functools  # Import functools to create a partial function
from scipy import interpolate
from lifelines import NelsonAalenFitter


# Locate project root and add to path
cwd = Path.cwd()
PROJECT_ROOT = next((p for p in [cwd, *cwd.parents] if (p / 'src').exists()), cwd)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Project-specific imports
from src.simulation import SR_sim
from src.plotting import SR_plotting
import src.sr_utils as utils
import src.correlation_analysis as ca
import src.twin_analysis as ta
import src.gamma_gompertz as gg
from src.HMD_lifetables import HMD
import saved_results.twin_studies_data as td
import saved_results.model_param_calibrations as pc

# Reproducibility setup
SEED = int(os.environ.get('PYTHONHASHSEED', '12345'))
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
RNG = np.random.default_rng(12345)

# Deterministic sampling helpers
def choice_deterministic(a, size=None, replace=True, p=None):
    return RNG.choice(a, size=size, replace=replace, p=p)

def rand_uniform(size=None):
    return RNG.random(size)

# Matplotlib configuration
mpl.rcParams['font.family'] = 'Arial'
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# Plot styling
plt.rc('axes', facecolor='white', grid=False)
plt.rc('axes.spines', top=False, right=False)
plt.rc('font', size=16)
plt.rc('axes', titlesize=28, labelsize=24)
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rc('xtick.major', size=8, width=2.5)
plt.rc('ytick.major', size=8, width=2.5)
plt.rc('xtick.minor', size=5, width=2.5)
plt.rc('ytick.minor', size=5, width=2.5)
plt.rc('legend', fontsize=16)
plt.rc('figure', titlesize=28)

def remove_top_right_spines(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

# Jupyter extensions
%load_ext autoreload
%autoreload 2


# Printing model fits

In [3]:
# Print calibrated parameters for all cohorts in a nice table format

print("=" * 80)
print("CALIBRATED MODEL PARAMETERS")
print("=" * 80)

cohorts = ['denmark', 'sweden', 'satsa', 'usa']

for cohort in cohorts:
    print(f"\n{cohort.upper()}")
    print("-" * 40)
    
    print("Modified Gamma-Gompertz (MGG) Model:")
    pc.print_calibrated_GG_params(cohort)
    
    print("\nStochastic Repair (SR) Model:")
    pc.print_calibrated_SR_params(cohort)
    
    if cohort != cohorts[-1]:  # Don't print separator after last cohort
        print("\n" + "=" * 40)

print("\n" + "=" * 80)

CALIBRATED MODEL PARAMETERS

DENMARK
----------------------------------------
Modified Gamma-Gompertz (MGG) Model:
a=1.00e-05 year⁻¹,
b=0.12 year⁻¹,
c = 30,
27% (23%,31%)
m = 2.20e-03 year⁻¹

Stochastic Repair (SR) Model:
η = 0.66 year⁻²,
β = 63.51 year⁻¹,
ε = 51.83 year⁻¹,
κ = 0.50,
Xc = 17.0,
21% (19%,23%)
m_ex = 3.00e-03 year⁻¹


SWEDEN
----------------------------------------
Modified Gamma-Gompertz (MGG) Model:
a=4.70e-06 year⁻¹,
b=0.12 year⁻¹,
c = 40,
31% (27%,35%)
m = 1.50e-03 year⁻¹

Stochastic Repair (SR) Model:
η = 0.66 year⁻²,
β = 64.06 year⁻¹,
ε = 51.83 year⁻¹,
κ = 0.50,
Xc = 18.4,
21% (20%,22%)
m_ex = 3.11e-03 year⁻¹


SATSA
----------------------------------------
Modified Gamma-Gompertz (MGG) Model:
a=4.23e-06 year⁻¹,
b=0.12 year⁻¹,
c = 20,
27% (24%,31%)
m = 1.00e-03 year⁻¹

Stochastic Repair (SR) Model:
η = 0.66 year⁻²,
β = 64.06 year⁻¹,
ε = 51.83 year⁻¹,
κ = 0.50,
Xc = 18.4,
20% (17%,23%)
m_ex = 2.00e-03 year⁻¹


USA
----------------------------------------
Modified Ga

In [5]:
sweden_cohort_f = HMD(country='swe', gender = 'female', data_type = 'cohort')
sweden_cohort_m = HMD(country='swe' , gender = 'male', data_type = 'cohort')
sweden_cohort_b = HMD(country='swe' , gender = 'both', data_type = 'cohort')

sweden_period_m = HMD(country='swe' , gender = 'male', data_type = 'period')
sweden_period_f = HMD(country='swe' , gender = 'female', data_type = 'period')

denmark_cohort_m = HMD(country='dan' , gender = 'male', data_type = 'cohort')
denmark_cohort_f = HMD(country='dan' , gender = 'female', data_type = 'cohort')
denmark_cohort_b = HMD(country='dan' , gender = 'both', data_type = 'cohort')

years = np.arange(1870, 1901, 5)



# Table of Results for Table 1

In [6]:
# Print h² correlation values for different cohorts at negligible mex (h_ext = 1e-30)

print("=== h² correlation values at negligible m_ex (h_ext = 1e-30) ===\n")

# Define the cohorts and their corresponding studies
cohort_studies = {
    'danish': td.herskind_study,
    'swedish': td.ljinquist_study, 
    'satsa': td.satsa_study,
    'usa': td.usa_study  # No specific study data available
}

# Collect data for table
table_data = []

for cohort_name, study in cohort_studies.items():
    row = [cohort_name.upper()]
    
    # SR at study's filter age
    if study is not None:
        filter_age = study['metadata']['filter_age']
        sr_corr_study_age = pc.get_correlation_value(
            model='SR', 
            filter_age=filter_age, 
            log10h_ext=-30, 
            metric='h2', 
            cohort=cohort_name
        )
        row.append(sr_corr_study_age)
    else:
        row.append("N/A")
    
    # MGG at study's filter age
    if study is not None:
        filter_age = study['metadata']['filter_age']
        mgg_corr_study_age = pc.get_correlation_value(
            model='GG', 
            filter_age=filter_age, 
            log10h_ext=-30, 
            metric='h2', 
            cohort=cohort_name
        )
        row.append(mgg_corr_study_age)
    else:
        row.append("N/A")
    
    # SR at filter age 15
    sr_corr_15 = pc.get_correlation_value(
        model='SR', 
        filter_age=15, 
        log10h_ext=-30, 
        metric='h2', 
        cohort=cohort_name
    )
    row.append(sr_corr_15)
    
    # MGG at filter age 15
    mgg_corr_15 = pc.get_correlation_value(
        model='GG', 
        filter_age=15, 
        log10h_ext=-30, 
        metric='h2', 
        cohort=cohort_name
    )
    row.append(mgg_corr_15)
    
    table_data.append(row)

# Print as formatted table
headers = ['Cohort', 'SR (Study Age)', 'MGG (Study Age)', 'SR (Age 15)', 'MGG (Age 15)']

from tabulate import tabulate
print(tabulate(table_data, headers=headers, tablefmt='grid'))


=== h² correlation values at negligible m_ex (h_ext = 1e-30) ===

+----------+-------------------+-------------------+-------------------+-------------------+
| Cohort   | SR (Study Age)    | MGG (Study Age)   | SR (Age 15)       | MGG (Age 15)      |
+==========+===================+===================+===================+===================+
| DANISH   | 0.57 (0.51, 0.63) | 0.47 (0.38, 0.54) | 0.57 (0.51, 0.63) | 0.47 (0.38, 0.54) |
+----------+-------------------+-------------------+-------------------+-------------------+
| SWEDISH  | 0.51 (0.50, 0.53) | 0.46 (0.40, 0.51) | 0.56 (0.55, 0.59) | 0.56 (0.49, 0.63) |
+----------+-------------------+-------------------+-------------------+-------------------+
| SATSA    | 0.44 (0.35, 0.50) | 0.33 (0.29, 0.38) | 0.53 (0.42, 0.62) | 0.47 (0.41, 0.56) |
+----------+-------------------+-------------------+-------------------+-------------------+
| USA      | 0.61 (0.56, 0.65) | 0.43 (0.37, 0.50) | 0.62 (0.57, 0.67) | 0.45 (0.38, 0.52) |
+---